# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

# Import the dataset
# mongoimport --db uk_food --collection establishments --file establishments.json


In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
client = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
# List all databases
databases = client.list_database_names()
print(databases)
# Confirm uk_food is listed
if 'uk_food' in databases:
    print("'uk_food' database exists!")
else:
    print("'uk_food' database does not exist.")

In [ ]:
# assign the uk_food database to a variable name
db = client['uk_food']



In [ ]:
# review the collections in our new database
collections = db.list_collection_names()

print("Collections in the 'uk_food' database:")
for collection_name in collections:
    print(f"- {collection_name}")

In [ ]:
# review a document in the establishments collection
document = establishments_collection.find_one()
pprint(document)

In [ ]:
# assign the collection to a variable
establishments_collection = db['establishments']


## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:

# Create a dictionary for the new restaurant data
penang_flavours = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}
print(penang_flavours)

In [ ]:
# Insert the new restaurant into the collection

result = establishments_collection.insert_one(penang_flavours)

In [ ]:
# Check that the new restaurant was inserted
inserted_document = establishments_collection.find_one({"_id": result.inserted_id})

if inserted_document:
    print("The new restaurant was successfully inserted!")
else:
    print("Failed to find the inserted restaurant.")

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type_data = establishments_collection.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"_id": 0, "BusinessTypeID": 1}
)

if not business_type_data:
    print("No matching BusinessType found.")
    exit()

business_type_id = business_type_data['BusinessTypeID']

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
update_result = establishments_collection.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": business_type_id}}
)


In [ ]:
# Confirm that the new restaurant was updated
if update_result.modified_count > 0:
    print("Successfully updated the BusinessTypeID for Penang Flavours!")
else:
    print("No changes were made.")

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
initial_count = establishments_collection.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of establishments in Dover: {initial_count}")

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments_collection.delete_many({"LocalAuthorityName": "Dover"})


In [ ]:
# Check if any remaining documents include Dover
final_count = establishments_collection.count_documents({})
deleted_count = initial_count - final_count
print(f"Number of documents after deletion: {final_count}")
print(f"Number of deleted establishments: {deleted_count}")

if deleted_count == initial_count:
    print("All Dover establishments were successfully deleted!")
else:
    print(f"{deleted_count} establishments were deleted. Some might still remain.")

In [ ]:
# Check that other documents remain with 'find_one'
document = establishments_collection.find_one()

if document:
    print("A document from the establishments collection:")
    pprint(document)
else:
    print("No documents remain in the establishments collection.")

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
cursor = establishments_collection.find(
    {
        "geocode.longitude": {"$type": "string"},
        "geocode.latitude": {"$type": "string"}
    }
)

for doc in cursor:
    # Convert string-based longitude and latitude to Decimal128
    longitude_decimal = Decimal128(doc["geocode"]["longitude"])
    latitude_decimal = Decimal128(doc["geocode"]["latitude"])

    # Update the document with the new Decimal128 values
    establishments_collection.update_one(
        {"_id": doc["_id"]},
         {
            "$set": {
                "geocode.longitude": longitude_decimal,
                "geocode.latitude": latitude_decimal
            }
        }
    )

print("Conversion from string to Decimal128 for longitude and latitude completed!")

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
result = establishments_collection.update_many(
    {"RatingValue": {"$type": "string"}},  # Filter documents with string-based RatingValue
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]  # Convert to integer
)

print(f"{result.modified_count} documents were updated.")

In [ ]:
# Check that the coordinates and rating value are now numbers
# Fetch a few sample documents
sample_documents = establishments_collection.find().limit(5)

# Check the data types of the specified fields for each sample document
for doc in sample_documents:
    longitude_type = type(doc.get("geocode", {}).get("longitude"))
    latitude_type = type(doc.get("geocode", {}).get("latitude"))
    rating_value_type = type(doc.get("RatingValue"))

    print(f"Document ID: {doc['_id']}")
    print(f"Longitude data type: {longitude_type}")
    print(f"Latitude data type: {latitude_type}")
    print(f"RatingValue data type: {rating_value_type}")
    print("-" * 50)